This code exports Climate features for the LUCAS samples<br>
Data: https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE<br>
Author: Nafiseh Kakhani, University of Tuebingen<br>
Date: 04/04/2023<br>
Project: Multi-modal deep learning for SOC estimation 

In [1]:
import ee
import geemap

In [2]:
# ee.Authenticate()

In [3]:
ee.Initialize()

In [4]:
#Import samples
region = ee.FeatureCollection("projects/ee-nfskhn/assets/LUCAS_2015") #Add your feature collection here
Desired_bands = ['aet', 'def', 'pdsi', 'pet', 'pr', 'ro', 'soil', 'srad', 'swe', 'tmmn', 'tmmx', 'vap', 'vpd', 'vs']
x = 0 #This needs to be inside a for-loop 
Climatefeature = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').select(Desired_bands[x]);
bandName = ee.Image(Climatefeature.first()).bandNames().get(0)
startDate = ee.Date('2010-01-01') # set analysis start time
endDate = ee.Date('2014-12-30') # set analysis end time

# calculate the number of months to process
nMonths = ee.Number(endDate.difference(startDate,'month')).round()

# get a list of time strings to pass into a dictionary later on
monList = ee.List(ee.List.sequence(0,nMonths).map(lambda n:
          startDate.advance(n,'month').format('YYYMMdd')))

# print(monList.getInfo())

def map_climate(feature):
    # map over each month
    def map_month(n):
        # calculate the offset from startDate
        ini = startDate.advance(n,'month')
        # advance just one month
        end = ini.advance(1,'month')
        # filter and reduce
        data = Climatefeature.filterDate(ini,end).mean().reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=1000
        )
        # get the value and check that it has data
        val = ee.Number(data.get(bandName))
        val = ee.Number(ee.Algorithms.If(val,val,-999))
        # return zonal mean
        return val
    timeSeries = ee.List.sequence(0,nMonths).map(map_month)
    # create new dictionary with date strings and values
    timeDict = ee.Dictionary.fromLists(monList,ee.List(timeSeries))
    # return feature with a timeseries property and results
    return feature.set(timeDict).setGeometry(None) # set the geometry to None to speed up the process

pointsList = region.toList(region.size())

#We need to break the large files in to chuncks 
chunk = 1000 #every 1000 points will be processed (to be checked) 
end = pointsList.size().getInfo()

#we set the end number of for loop manually
for i in range(0, end, chunk):
            subset = ee.FeatureCollection(region.toList(chunk, i)).map(map_climate)
            name = f"{Desired_bands[x]}_{i}.csv"
            geemap.ee_to_csv(subset, filename = name)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_0.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_2000.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_4000.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_6000.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_8000.csv
Generating URL ...
Please wait ...
Data downloaded to C:\Users\nkakhani\SoilSpec\aet_10000.csv
Generating URL ...
Please wait ...
An error occurred while downloading. 
 Retrying ...
Gen